In [148]:
# If you use a library for RSA probable message attack may not work as they use salt which gives different cipher each time
class RSA:
    def __init__(self):
        self.p = 211
        self.q = 719
        self.k = 14

        self.n = self.p * self.q     # Must be greater than ascii of each character to be encrypted otherwise decryption won't be same
        self.phi = (self.p-1) * (self.q-1)

        self.e = self._get_e(self.phi)
        self.d = self._get_d()

    # Help function to find e
    def _gcd(self, a, b):

        while b != 0:
            a, b = b, a % b

        return a

    
    # Finding e
    def _get_e(self, phi):
        e = 2

        while self._gcd(e, phi) != 1:   # It breaks if e is co-factor of phi
            e = e + 1

        return e


    # Now finding d
    def _get_d(self, init_val = 1):

        d = init_val

        while (self.e*d % self.phi) != 1:
            d = d + 1

        return d
    
    
    # Encrypting whole message using public key (e, n)
    def encrypt(self, message):
        cipher = []
        
        for ch in message:
            ch = ord(ch)   # Getting ASCII
            encrypted = ch**self.e % self.n
            cipher.append(encrypted)
        
        cipher = list( map(lambda enc_ch: str(enc_ch), cipher) )   
        return ','.join(cipher)
        
    
    # Decrypting a cipher using private key (d)
    def decrypt(self, cipher):
        cipher = cipher.split(',')
        message = []
        
        for ch in cipher:
            ch = int(ch)   
            decrypted = ch**self.d % self.n
            message.append(chr(decrypted))
        
        return ''.join(message)

In [147]:
# Probable Message Attack for a RSA encryption transmitting 56 bit DES key
class ProbableMessageAttack:
    def __init__(self, possible_key_byte_vals, rsa, cipher):
        self.possible_key_byte_vals = possible_key_byte_vals
        self.possible_des_keys = self.gen_possible_des_keys(possible_key_byte_vals)
        self.cipher = cipher
        
    
    # Method used for generating key one by one upon requirement
    def gen_possible_des_keys(self, k_vals):
    # Loops are 7 times because we are sayting that key must have 7 bytes only (56 bits) and each byte can be any value from k_vals
        for one in k_vals:
            for two in k_vals:
                for three in k_vals:
                    for four in k_vals:
                        for five in k_vals:
                            for six in k_vals:
                                for seven in k_vals:
                                    yield one+two+three+four+five+six+seven
                                        
    # Method to start brute forcing
    def start_attack(self):
        keys_searched = 0
        
        for des_key in self.possible_des_keys:
            keys_searched += 1
            print("des_key:", des_key)
            
            des_key_rsa_cipher = rsa.encrypt(des_key)
            print("Actual cipher:", self.cipher, "Generated cipher:", des_key_rsa_cipher, '\n')
            
            if des_key_rsa_cipher == self.cipher:
                print("DES Key found:", des_key, 'by brute forcing', keys_searched, 'keys')
                return des_key

In [154]:
rsa = RSA()
message = '1111115'  # Message to be transmitted is DES key thats why probable message attack can be performed

cipher = rsa.encrypt(message)
print('cipher:', cipher)

decrypted = rsa.decrypt(cipher)
print('decrypted:', decrypted)

cipher: 82116,82116,82116,82116,82116,82116,144613
decrypted: 1111115


In [155]:
possible_key_vals = ['1', '2', '3', '4', '5', '6', '7']   # Originally there are 56 bits in DES key so 7 bytes so each byte should have any of 256 possible values but that would take so much time
probable_message_attack = ProbableMessageAttack(possible_key_vals, rsa, cipher)
des_key_transmitted = probable_message_attack.start_attack()
print(des_key_transmitted)

des_key: 1111111
Actual cipher: 82116,82116,82116,82116,82116,82116,144613 Generated cipher: 82116,82116,82116,82116,82116,82116,82116 

des_key: 1111112
Actual cipher: 82116,82116,82116,82116,82116,82116,144613 Generated cipher: 82116,82116,82116,82116,82116,82116,50386 

des_key: 1111113
Actual cipher: 82116,82116,82116,82116,82116,82116,144613 Generated cipher: 82116,82116,82116,82116,82116,82116,27486 

des_key: 1111114
Actual cipher: 82116,82116,82116,82116,82116,82116,144613 Generated cipher: 82116,82116,82116,82116,82116,82116,57894 

des_key: 1111115
Actual cipher: 82116,82116,82116,82116,82116,82116,144613 Generated cipher: 82116,82116,82116,82116,82116,82116,144613 

DES Key found: 1111115 by brute forcing 5 keys
1111115
